In [0]:
secret=dbutils.secrets.get('databricklogin','databricklogin')

In [0]:
application_id ="88dec17f-62f0-4cab-82b7-b9022c2cc21c"
service_credential=secret
directory_id="037e0179-cd0e-4201-951c-167b7554d77a"
spark.conf.set("fs.azure.account.auth.type.datastoragenew12.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.datastoragenew12.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.datastoragenew12.dfs.core.windows.net", application_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.datastoragenew12.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.datastoragenew12.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
patients_df = spark.read.format("delta").load(cleaned_patient_output_path)
tests_df = spark.read.format("delta").load(cleaned_test_output_path)

# Join the cleaned patient and test data on PatientID
joined_df = patients_df.join(tests_df, on="PatientID", how="inner")

# Add a new column to indicate if a patient's test result is 'Positive' or 'Negative'
joined_df = joined_df.withColumn("ResultStatus", when(col("COVIDStatus") == "Positive", lit("Positive"))
                                               .otherwise(lit("Negative")))

# Select and transform relevant columns for reporting
etl_output_df = joined_df.select(
    col("PatientID"),
    col("Age"),
    col("COVIDStatus"),
    col("TestID"),
    col("LabName"),
    col("ResultDate"),
    col("ResultStatus")
)

# Show a preview of the transformed data
etl_output_df.show()

# Save the transformed data to Delta format in Azure Storage
etl_output_path = "abfss://gold@httpforstorage.dfs.core.windows.net/ETLResultData"

etl_output_df.write.format("delta").mode("append").save(etl_output_path)
print(f"ETL output data saved to {etl_output_path}")